#Data Labeling



##Libraries and connect to drive

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
import re

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Normal Data Labeling

###LOGS

In [ ]:
file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/all.log'

with open(file_path, 'r') as file:
    log_messages = file.readlines()

df_all = pd.DataFrame(log_messages, columns=['Message'])

df_all['Label'] = 0
df_all['Attack Technique'] = 0


df_all.to_csv('/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/all_logs_label.csv', index=False)


In [ ]:
import re
import pandas as pd

# Define file paths
log_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/all.log'

# Define log patterns
network_pattern = re.compile(r'Inbound Connection:|Outbound Connection:|SRC=|DST=|PROTO=TCP|PROTO=UDP')

# Initialize lists to store logs
event_logs = []
network_logs = []

# Process and separate logs into event and network logs
with open(log_file_path, 'r') as file:
    for line in file:
        if network_pattern.search(line):
            network_logs.append(line)
        else:
            event_logs.append(line)

# Load the separated logs and label them into pandas DataFrames
# For Event Logs
df_event = pd.DataFrame(event_logs, columns=['Message'])
df_event['Label'] = 0  # Label for event logs
df_event['Attack Technique'] = 0  # Default attack technique

# Save the labeled event logs as CSV
df_event.to_csv('/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/event_logs_label.csv', index=False)

# For Network Logs
df_network = pd.DataFrame(network_logs, columns=['Message'])
df_network['Label'] = 0  # Label for network logs
df_network['Attack Technique'] = 0  # Default attack technique

# Save the labeled network logs as CSV
df_network.to_csv('/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/network_logs_label.csv', index=False)


In [ ]:
file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Output.log'

with open(file_path, 'r') as file:
    log_messages = file.readlines()

df_output = pd.DataFrame(log_messages, columns=['Message'])

df_output['Label'] = 0
df_output['Attack Technique'] = 0

df_output.to_csv('/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/output_label.csv', index=False)

In [ ]:
import pandas as pd
import re

file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/system_logs.txt'
with open(file_path, 'r') as file:
    log_messages = file.readlines()

logs_data = []

memory_pattern = r"Memory - Total: \d+MB, Used: (\d+)MB, Free: \d+MB"
cpu_pattern = r"CPU Load - 1 min: ([\d\.]+), 5 min: ([\d\.]+), 15 min: ([\d\.]+)"
disk_pattern = r"Disk Space - Total: \d+M, Used: \d+M, Available: \d+M, Usage: (\d+)%"
network_pattern = r"Network - IP Address: [\d\.]+, TX: (\d+)MB, RX: (\d+)MB"
ports_pattern = r"Open Ports: ([\d, ]+)"


def process_log_chunk(chunk):
    current_data = {}
    current_data['Timestamp'] = chunk[-1].split(']')[0][5:]

    current_data['Used Memory (MB)'] = None
    current_data['CPU Load 1 min'] = None
    current_data['CPU Load 5 min'] = None
    current_data['CPU Load 15 min'] = None
    current_data['Disk Usage (%)'] = None
    current_data['TX (MB)'] = None
    current_data['RX (MB)'] = None
    current_data['Open Ports'] = None

    for message in chunk:

        memory_match = re.search(memory_pattern, message)
        if memory_match:
            current_data['Used Memory (MB)'] = int(memory_match.group(1))

        cpu_match = re.search(cpu_pattern, message)
        if cpu_match:
            current_data['CPU Load 1 min'] = float(cpu_match.group(1))
            current_data['CPU Load 5 min'] = float(cpu_match.group(2))
            current_data['CPU Load 15 min'] = float(cpu_match.group(3))

        disk_match = re.search(disk_pattern, message)
        if disk_match:
            current_data['Disk Usage (%)'] = int(disk_match.group(1))

        network_match = re.search(network_pattern, message)
        if network_match:
            current_data['TX (MB)'] = int(network_match.group(1))
            current_data['RX (MB)'] = int(network_match.group(2))

        ports_match = re.search(ports_pattern, message)
        if ports_match:
            current_data['Open Ports'] = ports_match.group(1)

    time_str = current_data['Timestamp'][11:16]
    current_data['Label'], current_data['Attack Technique'] = 0, 0

    return current_data

chunk = []
for message in log_messages:
    if "[UTC" in message and chunk:
        logs_data.append(process_log_chunk(chunk))
        chunk = []
    chunk.append(message)

if chunk:
    logs_data.append(process_log_chunk(chunk))

df_logs = pd.DataFrame(logs_data)

columns_to_check = [
    'Used Memory (MB)', 'CPU Load 1 min', 'CPU Load 5 min',
    'CPU Load 15 min', 'Disk Usage (%)', 'TX (MB)', 'RX (MB)', 'Open Ports'
]


df_logs = df_logs.dropna(subset=columns_to_check, how='all')


df_logs['Timestamp'] = pd.to_datetime(df_logs['Timestamp'])

df_combined = df_logs.groupby('Timestamp').agg({
    'Used Memory (MB)': 'first',
    'CPU Load 1 min': 'first',
    'CPU Load 5 min': 'first',
    'CPU Load 15 min': 'first',
    'Disk Usage (%)': 'first',
    'TX (MB)': 'first',
    'RX (MB)': 'first',
    'Open Ports': lambda x: ', '.join(x.dropna()),
    'Label': 'first',
    'Attack Technique': 'first'
}).reset_index()

output_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/system_label.csv'
df_combined.to_csv(output_file_path, index=False)

<ipython-input-4-79eb58559fad>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_logs['Timestamp'] = pd.to_datetime(df_logs['Timestamp'])


In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Timestamp         656 non-null    datetime64[ns]
 1   Used Memory (MB)  656 non-null    float64       
 2   CPU Load 1 min    656 non-null    float64       
 3   CPU Load 5 min    656 non-null    float64       
 4   CPU Load 15 min   656 non-null    float64       
 5   Disk Usage (%)    656 non-null    float64       
 6   TX (MB)           656 non-null    float64       
 7   RX (MB)           656 non-null    float64       
 8   Open Ports        656 non-null    object        
 9   Label             656 non-null    int64         
 10  Attack Technique  656 non-null    int64         
dtypes: datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 56.5+ KB


###SQLITE

In [ ]:
import sqlite3
import pandas as pd

db_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/default.sqlite'

conn = sqlite3.connect(db_path)

df = pd.read_sql_query("SELECT * FROM notifications", conn)

df['Message'] = df['timestamp'].astype(str) + ' ' + df['notificationName'] + ' ' + df['senderName'] + ' ' + \
                df['severity'].astype(str) + ' ' + df['payloadString'] + ' ' + df['payloadXml'] + ' ' + \
                df['redundancyType'] + ' ' + df['redundancyRole']

df['Label'] = 0
df['Attack Technique'] = 0

df_default = df[['Message', 'Label', 'Attack Technique']]


df_default.to_csv('/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/default_labeled.csv', index=False)

conn.close()

df_default.head()

,Message,Label,Attack Technique
0,2024-08-28T17:18:11.828773 Arp.System.Acf.Syst...,0,0
1,2024-08-28T17:18:11.829080 Arp.System.Acf.Syst...,0,0
2,2024-08-28T17:18:22.944492 Arp.Io.Axioline.Sta...,0,0
3,2024-08-28T17:18:25.187051 Arp.System.Acf.Syst...,0,0
4,2024-08-28T17:18:26.998054 Arp.Io.PnD.PnStatio...,0,0


In [ ]:
db_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/sec_loggs.db'

conn = sqlite3.connect(db_path)

query = 'SELECT * FROM log_all_2903489507'

df = pd.read_sql_query(query, conn)

df['datetime'] = pd.to_datetime(
    df['Arp.Plc.Eclr/code.RTC_DT.YEAR'].astype(str) + '-' +
    df['Arp.Plc.Eclr/code.RTC_DT.MONTH'].astype(str).str.zfill(2) + '-' +
    df['Arp.Plc.Eclr/code.RTC_DT.DAY'].astype(str).str.zfill(2) + ' ' +
    df['Arp.Plc.Eclr/code.RTC_DT.HOURS'].astype(str).str.zfill(2) + ':' +
    df['Arp.Plc.Eclr/code.RTC_DT.MINUTES'].astype(str).str.zfill(2) + ':' +
    df['Arp.Plc.Eclr/code.RTC_DT.SECONDS'].astype(str).str.zfill(2)
)

df_data_logger = df

df_data_logger['Total_Exec_Duration'] = (
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[1].TASK_INFOS[1].LAST_EXEC_DURATION'] +
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[2].TASK_INFOS[1].LAST_EXEC_DURATION']
)

df_data_logger['Total_Activation_delay'] = (
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[1].TASK_INFOS[1].LAST_ACTIVATION_DELAY'] +
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[2].TASK_INFOS[1].LAST_ACTIVATION_DELAY']
)

csv_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/datalogger_labeled.csv'

df_data_logger.to_csv(csv_path, index=False)

conn.close()

df_data_logger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67000 entries, 0 to 66999
Data columns (total 33 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   Timestamp                                                                     67000 non-null  int64         
 1   ConsistentDataSeries                                                          67000 non-null  int64         
 2   Arp.Plc.Eclr/code.AXIO_ACT                                                    67000 non-null  int64         
 3   Arp.Plc.Eclr/code.AXIO_BUS                                                    67000 non-null  int64         
 4   Arp.Plc.Eclr/code.AXIO_PF                                                     67000 non-null  int64         
 5   Arp.Plc.Eclr/code.AXIO_PW                                                     67000 non-

In [ ]:
df_data_logger.head()

,Timestamp,ConsistentDataSeries,Arp.Plc.Eclr/code.AXIO_ACT,Arp.Plc.Eclr/code.AXIO_BUS,Arp.Plc.Eclr/code.AXIO_PF,Arp.Plc.Eclr/code.AXIO_PW,Arp.Plc.Eclr/code.AXIO_RDY,Arp.Plc.Eclr/code.AXIO_RUN,Arp.Plc.Eclr/code.AXIO_SYSFAIL,Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[1].TASK_INFOS[1].LAST_ACTIVATION_DELAY,...,Arp.Plc.Eclr/code.USER_PART_DT.MEM_TOTAL,Arp.Plc.Eclr/code.USER_PART_DT.MEM_USAGE,Arp.Plc.Eclr/code.USER_PART_DT.MEM_USED,Arp.Plc.Eclr/code.cpu_1,Arp.Plc.Eclr/code.cpu_2,Arp.Plc.Eclr/code.temp,RecordAttribute,datetime,Total_Exec_Duration,Total_Activation_delay
0,5250291498608783304,1,0,0,0,0,0,0,0,0,...,511647744,27,126058496,98,73,55,1,2024-08-29 17:06:57,0,0
1,5250291498609782004,1,0,0,0,0,0,0,0,73,...,511647744,26,125505536,47,99,55,1,2024-08-29 17:06:58,441,146
2,5250291498610782554,1,0,0,0,0,0,0,0,106,...,511647744,26,125505536,47,99,55,1,2024-08-29 17:06:58,374,187
3,5250291498611781774,1,0,0,0,0,0,0,0,116,...,511647744,26,125505536,47,99,55,1,2024-08-29 17:06:58,504,190
4,5250291498612798694,1,0,0,0,0,0,0,0,50,...,511647744,26,125505536,47,99,55,1,2024-08-29 17:06:58,349,119


In [ ]:
import sqlite3
import pandas as pd

db_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/security.sqlite'

conn = sqlite3.connect(db_path)

df = pd.read_sql_query("SELECT * FROM notifications", conn)

df['Message'] = df['timestamp'].astype(str) + ' ' + df['notificationName'] + ' ' + df['senderName'] + ' ' + \
                df['severity'].astype(str) + ' ' + df['payloadString'] + ' ' + df['payloadXml'] + ' ' + \
                df['redundancyType'] + ' ' + df['redundancyRole']

df['Label'] = 0
df['Attack Technique'] = 0

# Select the relevant columns
df_security_sql = df[['Message', 'Label', 'Attack Technique']]


df_security_sql.to_csv('/content/drive/MyDrive/Master/R2_Machine_Learning/Normal/Labeled/security_labeled.csv', index=False)

conn.close()

df_security_sql.head()

,Message,Label,Attack Technique
0,2024-08-28T17:18:23.284854 Security.Arp.Device...,0,0
1,2024-08-28T17:18:23.285615 Security.Arp.Device...,0,0
2,2024-08-28T17:18:23.286117 Security.Arp.Device...,0,0
3,2024-08-28T17:18:23.286484 Security.Arp.Device...,0,0
4,2024-08-28T17:18:24.226691 Security.Arp.Servic...,0,0


##Attack Labeling

###Output.log


####This when you upload the original code cause it had logs from previous days

In [ ]:
"""log_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Output.log'

with open(log_file_path, 'r') as file:
    log_lines = file.readlines()

part1 = []
part2 = []

found_split = False

time_pattern = re.compile(r'24\.08\.24 21:45:\d{2}\.\d{3}')

for line in log_lines:
    if found_split or time_pattern.search(line):
        part2.append(line)
        found_split = True
    else:
        part1.append(line)

with open('/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Output.log', 'w') as file2:
    file2.writelines(part2)
"""

Log file has been split and saved.


In [ ]:
file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Output.log'

with open(file_path, 'r') as file:
    log_messages = file.readlines()

df_output_a = pd.DataFrame(log_messages, columns=['Message'])

df_output_a['Label'] = 0
df_output_a['Attack Technique'] = 0

df_output_a['Timestamp'] = df_output_a['Message'].apply(lambda x: re.match(r'(\d{2}\.\d{2}\.\d{2} \d{2}:\d{2}:\d{2}\.\d{3})', x).group(0))

df_output_a['Timestamp'] = pd.to_datetime(df_output_a['Timestamp'], format='%d.%m.%y %H:%M:%S.%f')

df_output_a.loc[df_output_a['Timestamp'] > pd.to_datetime('24.08.24 23:01:00.000', format='%d.%m.%y %H:%M:%S.%f'), ['Label', 'Attack Technique']] = [1, 5]

specific_log_time = pd.to_datetime('24.08.24 23:51:42.790', format='%d.%m.%y %H:%M:%S.%f')

df_output_a.loc[df_output_a['Timestamp'] >= specific_log_time, 'Attack Technique'] = 10

print(df_output_a.head())

df_output_a.to_csv('/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/Output_labeled.csv', index=False)


                                             Message  Label  Attack Technique  \
0  24.08.24 21:45:42.577 Arp.System.Acf.Internal....      0                 0   
1  24.08.24 21:45:42.590 CommonRemoting          ...      0                 0   
2  24.08.24 21:45:42.594 CommonRemoting          ...      0                 0   
3  24.08.24 21:45:42.594 CommonRemoting          ...      0                 0   
4  24.08.24 21:45:42.616 CommonRemoting          ...      0                 0   

                Timestamp  
0 2024-08-24 21:45:42.577  
1 2024-08-24 21:45:42.590  
2 2024-08-24 21:45:42.594  
3 2024-08-24 21:45:42.594  
4 2024-08-24 21:45:42.616  


###all.log

First we split the dataset into network related logs and other logs. We had many network logs so this approach made the manual logging esier.


In [ ]:
log_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/all.log'

event_log_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/experiment/filtered_all.log'
network_log_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/experiment/network_logs.log'

network_pattern = re.compile(r'Inbound Connection:|Outbound Connection:|SRC=|DST=|PROTO=TCP|PROTO=UDP')

event_logs = []
network_logs = []

with open(log_file_path, 'r') as file:
    for line in file:
        if network_pattern.search(line):
            network_logs.append(line)
        else:
            event_logs.append(line)

with open(event_log_path, 'w') as file:
    file.writelines(event_logs)

with open(network_log_path, 'w') as file:
    file.writelines(network_logs)


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/experiment/filtered_all.log'
with open(file_path, 'r') as file:
    log_messages = file.readlines()

df_all = pd.DataFrame(log_messages, columns=['Message'])

df_all['Label'] = 0
df_all['Attack Technique'] = 0

malicious_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/experiment/malicious_logs.txt'
with open(malicious_file_path, 'r') as file:
    malicious_logs = file.readlines()

attack_technique = 2
label = 1

for line in malicious_logs:
    line = line.strip()

    if line == "":
        attack_technique += 1
    else:
        found_idx = df_all[df_all['Message'].str.strip() == line].index

        if not found_idx.empty:
            df_all.loc[found_idx, 'Label'] = label
            df_all.loc[found_idx, 'Attack Technique'] = attack_technique
        else:
            new_row = pd.DataFrame({'Message': [line], 'Label': [label], 'Attack Technique': [attack_technique]})
            df_all = pd.concat([df_all, new_row], ignore_index=True)

df_all.loc[df_all['Message'].str.contains('linpease.sh'), ['Label', 'Attack Technique']] = [1, 2]

df_all.loc[df_all['Message'].str.contains('private_file.txt.swp'), ['Label', 'Attack Technique']] = [1, 4]

output_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/all_logs_labeled.csv'
df_all.to_csv(output_file_path, index=False)

print(f"Processed logs have been saved to {output_file_path}")


Processed logs have been saved to /content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/all_logs_labeled.csv


In [ ]:
import pandas as pd
import re

file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/experiment/network_logs.log'
with open(file_path, 'r') as file:
    log_messages = file.readlines()

df_network = pd.DataFrame(log_messages, columns=['Message'])

df_network['Label'] = 0
df_network['Attack Technique'] = 0

ip_addresses = ['192.168.137.50', '192.168.137.246']
special_ips = ['192.168.137.12']

time_ranges = [
    ("21:59", "22:02", 1),
    ("22:03", "22:24", 2),
    ("22:25", "22:27", 3),
    ("22:28", "22:59", 4),
    ("23:00", "23:09", 5),
    ("23:10", "23:19", 6),
    ("23:20", "23:22", 7),
    ("23:23", "23:28", 8),
    ("23:29", "23:50", 9),
    ("23:51", "23:59", 10),
    ("00:00", "00:01", 10),
    ("00:02", "00:08", 11)
]

special_time_range = ("23:20", "23:22")

def contains_ip(message, ips):
    return any(ip in message for ip in ips)

def get_attack_technique(timestamp, ranges):
    for start, end, technique in ranges:
        if start <= timestamp <= end:
            return technique
    return 0

def check_special_conditions(message, timestamp, special_time_range):
    if special_time_range[0] <= timestamp <= special_time_range[1]:
        if contains_ip(message, special_ips) and "PT=22" in message:
            return True
    return False

for i, row in df_network.iterrows():
    message = row['Message']

    timestamp_match = re.search(r'\b\d{2}:\d{2}\b', message)
    if timestamp_match:
        timestamp = timestamp_match.group(0)

        if check_special_conditions(message, timestamp, special_time_range):
            df_network.at[i, 'Label'] = 1
            df_network.at[i, 'Attack Technique'] = 7
        elif contains_ip(message, ip_addresses):
            df_network.at[i, 'Label'] = 1
            df_network.at[i, 'Attack Technique'] = get_attack_technique(timestamp, time_ranges)

output_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/all_network_logs_labeled.csv'
df_network.to_csv(output_file_path, index=False)

print(f"Processed network logs have been saved to {output_file_path}")


Processed network logs have been saved to /content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/all_network_logs_labeled.csv


###system_logs


In [ ]:
import pandas as pd
import re

file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/system_logs.txt'
with open(file_path, 'r') as file:
    log_messages = file.readlines()

logs_data = []

memory_pattern = r"Memory - Total: \d+MB, Used: (\d+)MB, Free: \d+MB"
cpu_pattern = r"CPU Load - 1 min: ([\d\.]+), 5 min: ([\d\.]+), 15 min: ([\d\.]+)"
disk_pattern = r"Disk Space - Total: \d+M, Used: \d+M, Available: \d+M, Usage: (\d+)%"
network_pattern = r"Network - IP Address: [\d\.]+, TX: (\d+)MB, RX: (\d+)MB"
ports_pattern = r"Open Ports: ([\d, ]+)"


time_ranges = [
    ("21:59", "22:02", 1),
    ("22:03", "22:24", 2),
    ("22:25", "22:27", 3),
    ("22:28", "22:59", 4),
    ("23:00", "23:09", 5),
    ("23:10", "23:19", 6),
    ("23:20", "23:22", 7),
    ("23:23", "23:28", 8),
    ("23:29", "23:50", 9),
    ("23:51", "23:59", 10),
    ("00:00", "00:01", 10),
    ("00:02", "00:08", 11)
]


def get_attack_technique_and_label(timestamp, ranges):
    for start, end, technique in ranges:
        if start <= timestamp <= end:
            return 1, technique
    return 0, 0

def process_log_chunk(chunk):
    current_data = {}
    current_data['Timestamp'] = chunk[-1].split(']')[0][5:]

    current_data['Used Memory (MB)'] = None
    current_data['CPU Load 1 min'] = None
    current_data['CPU Load 5 min'] = None
    current_data['CPU Load 15 min'] = None
    current_data['Disk Usage (%)'] = None
    current_data['TX (MB)'] = None
    current_data['RX (MB)'] = None
    current_data['Open Ports'] = None

    for message in chunk:
        memory_match = re.search(memory_pattern, message)
        if memory_match:
            current_data['Used Memory (MB)'] = int(memory_match.group(1))

        cpu_match = re.search(cpu_pattern, message)
        if cpu_match:
            current_data['CPU Load 1 min'] = float(cpu_match.group(1))
            current_data['CPU Load 5 min'] = float(cpu_match.group(2))
            current_data['CPU Load 15 min'] = float(cpu_match.group(3))

        disk_match = re.search(disk_pattern, message)
        if disk_match:
            current_data['Disk Usage (%)'] = int(disk_match.group(1))

        network_match = re.search(network_pattern, message)
        if network_match:
            current_data['TX (MB)'] = int(network_match.group(1))
            current_data['RX (MB)'] = int(network_match.group(2))

        ports_match = re.search(ports_pattern, message)
        if ports_match:
            current_data['Open Ports'] = ports_match.group(1)

    time_str = current_data['Timestamp'][11:16]
    current_data['Label'], current_data['Attack Technique'] = get_attack_technique_and_label(time_str, time_ranges)

    return current_data

chunk = []
for message in log_messages:
    if "[UTC" in message and chunk:
        logs_data.append(process_log_chunk(chunk))
        chunk = []
    chunk.append(message)

if chunk:
    logs_data.append(process_log_chunk(chunk))

df_logs = pd.DataFrame(logs_data)

columns_to_check = [
    'Used Memory (MB)', 'CPU Load 1 min', 'CPU Load 5 min',
    'CPU Load 15 min', 'Disk Usage (%)', 'TX (MB)', 'RX (MB)', 'Open Ports'
]


df_logs = df_logs.dropna(subset=columns_to_check, how='all')


df_logs['Timestamp'] = pd.to_datetime(df_logs['Timestamp'])

df_combined = df_logs.groupby('Timestamp').agg({
    'Used Memory (MB)': 'first',
    'CPU Load 1 min': 'first',
    'CPU Load 5 min': 'first',
    'CPU Load 15 min': 'first',
    'Disk Usage (%)': 'first',
    'TX (MB)': 'first',
    'RX (MB)': 'first',
    'Open Ports': lambda x: ', '.join(x.dropna()),
    'Label': 'first',
    'Attack Technique': 'first'
}).reset_index()

output_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/system_logs_labeled.csv'
df_combined.to_csv(output_file_path, index=False)


###Default.sqlite

In [ ]:
import sqlite3

def safe_decode(value):
    try:
        if isinstance(value, bytes):
            return value.decode('utf-8', errors='ignore')
        return str(value)
    except (UnicodeDecodeError, AttributeError):
        return str(value)

database_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/default.sqlite'
conn = sqlite3.connect(database_path)

conn.text_factory = bytes

cursor = conn.cursor()
cursor.execute("SELECT * FROM notifications")

rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]

log_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/default.log'

with open(log_file_path, 'w') as file:
    for row in rows:
        decoded_row = [safe_decode(value) for value in row]
        log_entry = (
            f"[ID: {decoded_row[0]}] "
            f"[Timestamp: {decoded_row[1]}] "
            f"[Notification Name: {decoded_row[2]}] "
            f"[Sender: {decoded_row[3]}] "
            f"[Severity: {decoded_row[4]}] "
            f"[Payload: {safe_decode(decoded_row[5])}] "
            f"[Redundancy Type: {decoded_row[6]}] "
            f"[Redundancy Role: {decoded_row[7]}] "
            f"[Payload XML: {safe_decode(decoded_row[8])}]"
            "\n"
        )
        file.write(log_entry)

conn.close()

In [ ]:
import pandas as pd
import re

log_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/default.log'

with open(log_file_path, 'r') as file:
    log_entries = file.readlines()

messages = []
labels = []
attack_techniques = []

pattern = re.compile(
    r'\[ID:\s(\d+)\]\s\[Timestamp:\s([\d\-T:.]+)\]\s\[Notification Name:\s([\w.]+)\]\s'
    r'\[Sender:\s([\w\s]+)\]\s\[Severity:\s([\w]+)\]\s\[Payload:\s([^\]]+)\]'
)

start_time = "2024-08-24T21:45:42.891779"
mid_time = "2024-08-24T23:07:39"
end_time = "2024-08-24T23:54:00"

for entry in log_entries:
    match = pattern.search(entry)
    if match:
        timestamp = match.group(2)

        if timestamp < start_time:
            continue

        message = f"[ID: {match.group(1)}] [Timestamp: {timestamp}] [Notification Name: {match.group(3)}] [Sender: {match.group(4)}] [Severity: {match.group(5)}] [Payload: {match.group(6)}]"
        messages.append(message)

        if timestamp < mid_time:
            labels.append(0)
            attack_techniques.append(0)
        elif timestamp <= end_time:
            labels.append(1)
            attack_techniques.append(5)
        else:
            labels.append(1)
            attack_techniques.append(10)

df = pd.DataFrame({
    'Message': messages,
    'Label': labels,
    'Attack Technique': attack_techniques
})

output_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/default_labeled.csv'
df.to_csv(output_file_path, index=False)


###Security_logs.sqlite

In [ ]:
import sqlite3
import pandas as pd


def safe_decode(value):
    try:
        return value.decode('utf-8', errors='ignore')
    except (UnicodeDecodeError, AttributeError):
        return str(value)


def determine_label_and_technique(timestamp):
    time_str = timestamp.split('T')[1][:5]
    if '23:05' <= time_str < '23:51':
        return 1, 5
    elif '23:51' <= time_str or time_str < '00:03':
        return 1, 10
    elif time_str >= '00:03':
        return 1, 11
    else:
        return 0, 0

database_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/security.sqlite'
conn = sqlite3.connect(database_path)

cursor = conn.cursor()
cursor.execute("SELECT * FROM notifications")

rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]

data = []

for row in rows:

    decoded_row = [safe_decode(value) for value in row]


    if decoded_row[1] > '2024-08-24T21:46:03':

        message = (
            f"[ID: {decoded_row[0]}] "
            f"[Timestamp: {decoded_row[1]}] "
            f"[Notification Name: {decoded_row[2]}] "
            f"[Sender: {decoded_row[3]}] "
            f"[Severity: {decoded_row[4]}] "
            f"[Payload: {decoded_row[5]}] "
            f"[Redundancy Type: {decoded_row[6]}] "
            f"[Redundancy Role: {decoded_row[7]}] "
            f"[Payload XML: {decoded_row[8]}]"
        )


        label, attack_technique = determine_label_and_technique(decoded_row[1])


        data.append([message, label, attack_technique])


df = pd.DataFrame(data, columns=['Message', 'Label', 'Attack Technique'])


csv_file_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/security_labeled.csv'
df.to_csv(csv_file_path, index=False)

conn.close()

###Datalogger

In [ ]:
import sqlite3
import pandas as pd

db_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/sec_loggs.db'

conn = sqlite3.connect(db_path)

query = 'SELECT * FROM log_all_2903489507'

df = pd.read_sql_query(query, conn)

df['datetime'] = pd.to_datetime(
    df['Arp.Plc.Eclr/code.RTC_DT.YEAR'].astype(str) + '-' +
    df['Arp.Plc.Eclr/code.RTC_DT.MONTH'].astype(str).str.zfill(2) + '-' +
    df['Arp.Plc.Eclr/code.RTC_DT.DAY'].astype(str).str.zfill(2) + ' ' +
    df['Arp.Plc.Eclr/code.RTC_DT.HOURS'].astype(str).str.zfill(2) + ':' +
    df['Arp.Plc.Eclr/code.RTC_DT.MINUTES'].astype(str).str.zfill(2) + ':' +
    df['Arp.Plc.Eclr/code.RTC_DT.SECONDS'].astype(str).str.zfill(2)
)

df_data_logger = df.copy()

df_data_logger['Total_Exec_Duration'] = (
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[1].TASK_INFOS[1].LAST_EXEC_DURATION'] +
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[2].TASK_INFOS[1].LAST_EXEC_DURATION']
)

df_data_logger['Total_Activation_delay'] = (
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[1].TASK_INFOS[1].LAST_ACTIVATION_DELAY'] +
    df_data_logger['Arp.Plc.Eclr/code.ESM_DATA1.ESM_INFOS[2].TASK_INFOS[1].LAST_ACTIVATION_DELAY']
)

df_data_logger['Label'] = 0
df_data_logger['Attack Technique'] = 0

attack_chain = [
    ("21:59", "22:02", 1),
    ("22:03", "22:24", 2),
    ("22:25", "22:27", 3),
    ("22:28", "22:59", 4),
    ("23:00", "23:09", 5),
    ("23:10", "23:19", 6),
    ("23:20", "23:22", 7),
    ("23:23", "23:28", 8),
    ("23:29", "23:50", 9),
    ("23:51", "23:59", 10),
    ("00:00", "00:01", 10),
    ("00:02", "00:08", 11)
]

def label_attack_chain(row, chain):
    time_str = row['datetime'].time().strftime("%H:%M")
    for start, end, label in chain:
        if end:
            if start <= time_str < end:
                return pd.Series([label, label])
        else:
            if start <= time_str:
                return pd.Series([label, label])
    return pd.Series([0, 0])

df_data_logger = df_data_logger.iloc[25100:].reset_index(drop=True)

df_data_logger[['Label', 'Attack Technique']] = df_data_logger.apply(label_attack_chain, axis=1, chain=attack_chain)

csv_path = '/content/drive/MyDrive/Master/R2_Machine_Learning/Attack/Labeled/datalogger_labeled.csv'

df_data_logger.to_csv(csv_path, index=False)

conn.close()
